In [ ]:

import joblib as jb
import pandas as pd
import numpy as np
import re
import os
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
import basty.project.experiment_processing as experiment_processing
import seaborn as sns
sns.set_theme(style = 'dark')
import datetime
from dateutil import parser
import matplotlib.dates as mdates

from pathlib import Path
from basty.utils import misc
from process_results import BehaviorData
from filter_flies import FilterData
import sys
sys.path.append("/home/grover/Desktop/gitDLC/basty")

In [ ]:
CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"
project = experiment_processing.Project(
   CONFIG_PATH
)
FPS = project.fps
FIG_PATH = r'C:\Users\Grover\Documents\GitHub\deepsleepfigs'

In [ ]:
data_path_dict = project.data_path_dict
df = misc.parse_experiment_names(data_path_dict)
flysexpath = r'Y:\DeepSleepPaperData\flysexinfo.yaml'
df = misc.update_expt_info_df(df,flysexpath)

In [ ]:
df

In [ ]:
llh = misc.get_likelihood(data_path_dict,CONFIG_PATH,n_workers=None)

In [ ]:
results_folder = r"Z:\mfk\basty-projects\backup-allnohaltere\results\semisupervised_pair_kNN\predictions.15NN.neighbor_weights-distance.neighbor_weights_norm-log_count.activation-standard.voting-soft.voting_weights-None\exports"

In [ ]:
csv_files = [file for file in os.listdir(results_folder) if file.endswith('.csv')]

In [ ]:
dfs = list()
for file in csv_files:
    data = pd.read_csv(os.path.join(results_folder,file))
    data['ExptNames'] = os.path.splitext(file)[0]
    dfs.append(data)

df_data = pd.concat(dfs, ignore_index = True)
df_data = df_data.rename(columns = {'Unnamed: 0': 'Idx'})

In [ ]:
#Remove unnecessary columns
df_data.drop(['HaltereSwitch','Noise'],axis=1,inplace=True)

In [ ]:
llh  = llh[llh['ExptNames'].isin(df_data.ExptNames.unique())]

In [ ]:
llh

In [ ]:
BODY_PART_SETS = {'ProboscisPumping':'prob'}

BEHAVIORS = [
'Idle&Other',
'PosturalAdjustment&Moving',
'Feeding',
'Grooming',
'ProboscisPumping']

In [ ]:
newDf = pd.merge(df_data,df,left_on = 'ExptNames',right_on = 'ExptNames')

In [ ]:
wt_all = newDf[newDf['SD']==False]
male_wt = wt_all[wt_all['Sex'] == 'M']
female_wt = wt_all[wt_all['Sex'] == 'F']

In [ ]:
data = BehaviorData(behaviors=BEHAVIORS)

In [ ]:
newDf

In [ ]:
newDf

In [ ]:
filterer = FilterData(newDf,llh)

In [ ]:
filterer.plot_mean_likelihood()

In [ ]:
binary_mask = filterer.create_binary_masks(llh, BODY_PART_SETS, 0.7)

In [ ]:
masked_data_df = filterer.apply_binary_masks(binary_mask,newDf)

In [ ]:
masked_data_df

In [ ]:
binary_mask

In [ ]:
masked_data_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is called 'masked_data_df'
# and has columns 'ExptNames', 'ValueColumn' (replace with your actual column name)
# Add row numbers if they don't exist
masked_data_df['Row'] = masked_data_df.index

# Pivot the DataFrame
pivoted_df = masked_data_df.pivot(index='ExptNames', columns='Row', values='ProboscisPumping')

# Create a heatmap using Seaborn, handling NaN values with a different color (e.g., gray)
plt.figure(figsize=(16, 9))
sns.heatmap(pivoted_df, cmap='coolwarm', linewidths=0.5, cbar_kws={'ticks': [0, 1]}, mask=pivoted_df.isna(), linecolor='gray')

# Set plot attributes
plt.xlabel('Rows')
plt.ylabel('ExptNames')
plt.title('Heatmap of Data')

# Save the plot as a PNG file
FIG_PATH = r'C:\Users\Grover\Documents\GitHub\deepsleepfigs'
plt.savefig(f"{FIG_PATH}\\heatmap.png", dpi=300, bbox_inches='tight')

# Display the plot in the notebook (optional)
plt.show()



In [ ]:
plot_folder = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs\score_plots"
if not os.path.isdir(plot_folder):
    os.makedirs(plot_folder)

In [ ]:
male_wt['Time'] = male_wt['Idx'].apply(BehaviorData.get_time_stamp)
male_wt.index = pd.to_datetime(list(male_wt["Time"]))
rate = '1S'
male_wt_rs = BehaviorData.resample_df(male_wt,rate,BEHAVIORS)
male_wt_rs['TimeStamp'] = male_wt_rs.index

In [ ]:
male_wt

In [ ]:
llh['Time'] = llh['Idx'].apply(BehaviorData.get_time_stamp)

In [ ]:
BehaviorData.pivot_and_plot(male_wt_rs,'mal_b_scr_',plot_folder,rate,BEHAVIORS)

In [ ]:
import os

In [ ]:
os.listdir(plot_folder)

In [ ]:
df_pivoted = male_wt_rs.pivot(index='ExptNames', columns='TimeStamp', values='ProboscisPumping')
a4_dims = (25.7, 5.27)
fig, ax = plt.subplots(figsize=a4_dims)
plt.title('ProboscisPumping')
ax = sns.heatmap(df_pivoted, cmap='YlGnBu')

In [ ]:
male_wt_rs

In [ ]:
male_wt_rs